In [194]:
import requests
from bs4 import BeautifulSoup
import re
import json

In [9]:
base_url = 'https://ppp.gov.ph/'
extensions = 'list-of-projects/'
url = f'{base_url}{extensions}'

In [12]:
response = requests.get(url)
page_text = response.text
soup = BeautifulSoup(page_text)

In [96]:
selector = 'body > div:nth-child(7) > div.large-9.medium-8.columns.center > div.su-tabs.su-tabs-style-default.my-custom-tabs > div.su-tabs-panes'
view_content = soup.select(selector)


# Getting links
spoiler_links = []

for part in view_content:
    for s in part.findAll('a'):
        try:
            link = s['href']
        except:
            pass
        spoiler_links.append(link)

In [182]:
print(len(spoiler_links))
print(spoiler_links)

234
['https://ppp.gov.ph/ppp_projects/development-operations-and-maintenance-of-general-santos-port/', 'https://ppp.gov.ph/ppp_projects/expansion-operation-and-maintenance-of-vessel-traffic-services-vts-in-seven-philippine-ports/', 'https://ppp.gov.ph/ppp_projects/road-transport-it-infrastructure-project-phase-ii/', 'https://ppp.gov.ph/ppp_projects/50-year-integrated-development-plan-for-mactan-cebu-international-airport-mcia-project/', 'https://ppp.gov.ph/ppp_projects/c5-mrt-10-project/', 'https://ppp.gov.ph/ppp_projects/cavite-tagaytay-batangas-expressway-project/', 'https://ppp.gov.ph/ppp_projects/cagayan-valley-medical-center-hemodialysis-unit/', 'https://ppp.gov.ph/ppp_projects/bghmc-renal-center-building-project/', 'https://ppp.gov.ph/ppp_projects/cebu-monorail-transit-system-project/', 'https://ppp.gov.ph/ppp_projects/davao-sasa-port/', 'https://ppp.gov.ph/ppp_projects/davao-people-mover/', 'https://ppp.gov.ph/ppp_projects/davao-international-airport-development-operation-and-ma

In [192]:
data = []

for url in spoiler_links:
    try:
        response = requests.get(url)
        page_text = response.text
        soup = BeautifulSoup(page_text)

        selector = 'body > div:nth-child(7) > div.large-9.medium-8.columns.center'
        view_content = soup.select(selector)[0]

        subdata = {}

        # Get the title
        title_selector = 'div.title-block > h2'
        title_element = view_content.select(title_selector)
        try:
            title = title_element[0].text
            title = ' '.join(title.split())
        except:
            title = None
            pass
        subdata['title'] = title

        # Get other data
        data_selector = 'div.su-accordion > div > div > div.su-spoiler-content.su-clearfix'
        view_data = view_content.select(data_selector)[0]

        # Get cost
        para_selector = 'p'
        para_element = view_data.select(para_selector)

        for p in para_element:
            if 'php' in p.text.lower():
                cost = p.text
                cost =  ' '.join(cost.split())

        subdata['cost'] = cost

        # Get Region
        region = []
        li_selector = 'li'
        li_element = view_data.select(li_selector)
        for l in li_element:
            if 'REGION' in str(l) or 'CARAGAN' in str(l):
                region.append(l.text)
        subdata['region'] = region

        data.append(subdata)
    except:
        print(f'Error with handling {url}')
        continue

Error with handling http://www.dpwh.gov.ph/dpwh/ppp/priority


In [196]:
with open('Final-Proj_Region-Cost.json','w') as j:
    json.dump(data, j, indent = 4)

In [184]:
# Webscrape 1 Site
# THIS IS NOT THE ACTUAL CODE THAT I RAN (just for me to trial if it works since it takes long to run all sites)
response = requests.get(spoiler_links[1])
page_text = response.text
soup = BeautifulSoup(page_text)

selector = 'body > div:nth-child(7) > div.large-9.medium-8.columns.center'
view_content = soup.select(selector)[0]

subdata = {}

# Get the title
title_selector = 'div.title-block > h2'
title_element = view_content.select(title_selector)
try:
    title = title_element[0].text
    title = ' '.join(title.split())
except:
    title = None
    pass
subdata['title'] = title

# Get other data
data_selector = 'div.su-accordion > div > div > div.su-spoiler-content.su-clearfix'
view_data = view_content.select(data_selector)[0]

# Get cost
para_selector = 'p'
para_element = view_data.select(para_selector)

for p in para_element:
    if 'php' in p.text.lower():
        cost = p.text
        cost =  ' '.join(cost.split())

subdata['cost'] = cost

# Get Region
region = []
li_selector = 'li'
li_element = view_data.select(li_selector)
for l in li_element:
    if 'REGION' in str(l) or 'CARAGAN' in str(l):
        region.append(l.text)
subdata['region'] = region

print(subdata)

{'title': 'Expansion, Operation and Maintenance of Vessel Traffic Services (VTS) in Seven Philippine Ports', 'cost': 'Php 1.448 Billion', 'region': ['REGION IV-A - CALABARZON', 'REGION IV-B - MIMAROPA', 'REGION VI - WESTERN VISAYAS', 'REGION X - NORTHERN MINDANAO', 'REGION XI - DAVAO', 'REGION XII - SOCCSKSARGEN', 'NATIONAL CAPITAL REGION']}
